In [11]:
import itertools
import json
 
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

In [8]:

with open("data/post_1.json", "r") as file_handler:    
    userdata = json.load(file_handler)
userdata

{'id': 1,
 'user_fullname': 'A person',
 'avatar_img': 'https://i.pravatar.cc/152',
 'date_text': '14 November 2021 at 19:28',
 'timestamp': '1636936130.866307',
 'message': 'Im sick of all the politically correct rhetoric going out there. Stop spewing words like Mad Men, Orange is the New Black',
 'has_img': True,
 'post_img': 'https://picsum.photos/401/300',
 'has_comments': True,
 'comments': [{'id': 1,
   'user_fullname': 'A friend',
   'time_ago': '14h',
   'avatar_img': 'https://i.pravatar.cc/153',
   'message': 'This is a comment'},
  {'id': 2,
   'user_fullname': 'Another friend',
   'time_ago': '12h',
   'avatar_img': 'https://i.pravatar.cc/148',
   'message': 'You are sick, man!'}]}

In [9]:

with open("data/post_w.json", "w") as file_handler:    
    userdata = json.dump(userdata, file_handler)

In [41]:
for post in userdata['posts']:
    msg = post['message'].split()
    print(findsubsets(msg, 2))

[('Im', 'sick'), ('Im', 'of'), ('Im', 'all'), ('Im', 'the'), ('Im', 'politically'), ('Im', 'correct'), ('Im', 'rhetoric'), ('Im', 'going'), ('Im', 'out'), ('Im', 'there.'), ('Im', 'Stop'), ('Im', 'spewing'), ('Im', 'words'), ('Im', 'like'), ('Im', 'Mad'), ('Im', 'Men,'), ('Im', 'Orange'), ('Im', 'is'), ('Im', 'the'), ('Im', 'New'), ('Im', 'Black'), ('sick', 'of'), ('sick', 'all'), ('sick', 'the'), ('sick', 'politically'), ('sick', 'correct'), ('sick', 'rhetoric'), ('sick', 'going'), ('sick', 'out'), ('sick', 'there.'), ('sick', 'Stop'), ('sick', 'spewing'), ('sick', 'words'), ('sick', 'like'), ('sick', 'Mad'), ('sick', 'Men,'), ('sick', 'Orange'), ('sick', 'is'), ('sick', 'the'), ('sick', 'New'), ('sick', 'Black'), ('of', 'all'), ('of', 'the'), ('of', 'politically'), ('of', 'correct'), ('of', 'rhetoric'), ('of', 'going'), ('of', 'out'), ('of', 'there.'), ('of', 'Stop'), ('of', 'spewing'), ('of', 'words'), ('of', 'like'), ('of', 'Mad'), ('of', 'Men,'), ('of', 'Orange'), ('of', 'is'), ('

In [4]:
data = "You and I are stupid!"

In [12]:
import numpy as np
def word_combinations(sentence):
    data = sentence.split()
    out = []
    for j in range(0, len(data)+1):
        for i in range(0, j):
            data0 = data.copy()
            words = data0[i:j]

            for w in words: data0.remove(w)
            out.append(w)
            out.append(" ".join(data0))
    
    #for i in range(len(out)):
    #    if len(out[i])> 1:
    #        out[i] = " ".join(out[i])
    out =  list(np.unique(out))
    out.remove('')
    return out


from detoxify import Detoxify

model = Detoxify('original', device='cpu')
words = word_combinations(data)
print(words)



NameError: name 'data' is not defined

In [6]:
preds = []

for word in words:
    pred = model.predict(word)
    preds.append(pred)
        

In [7]:
pred_0 = {'toxicity':0, 'severe_toxicity':0, 'obscene':0, 'threat': 0, 'insult':0, 'identity_hate':0}
scores = {}
for word in data.split():
    scores[word] = pred_0.copy()

for word, pred in zip(words, preds):
    #print(word)
    for w in word.split():
        for cat in pred_0.keys():
            scores[w][cat] += pred[cat]/len(words)
            #print(w, cat, pred[cat])
    


In [1]:

import pandas as pd
df = pd.DataFrame(scores)


print(data, model.predict(data))
print(df)
print(df.sum(axis=0))

NameError: name 'scores' is not defined

In [9]:
model.predict('i hate you')

{'toxicity': 0.95099854,
 'severe_toxicity': 0.007945027,
 'obscene': 0.023724578,
 'threat': 0.039169792,
 'insult': 0.137607,
 'identity_hate': 0.02070892}

In [10]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(
    subset='train',
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=('headers', 'footers'),
)
twenty_test = fetch_20newsgroups(
    subset='test',
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=('headers', 'footers'),
)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline, make_pipeline

vec = TfidfVectorizer(min_df=3, stop_words='english',
                      ngram_range=(1, 2))
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
lsa = make_pipeline(vec, svd)

clf = SVC(C=150, gamma=2e-2, probability=True)
pipe = make_pipeline(lsa, clf)
pipe.fit(twenty_train.data, twenty_train.target)
pipe.score(twenty_test.data, twenty_test.target)


0.8901464713715047

In [12]:
def print_prediction(doc):
    y_pred = pipe.predict_proba([doc])[0]
    for target, prob in zip(twenty_train.target_names, y_pred):
        print("{:.3f} {}".format(prob, target))

doc = twenty_test.data[0]
print_prediction(doc)

0.001 alt.atheism
0.001 comp.graphics
0.996 sci.med
0.003 soc.religion.christian


In [13]:
import eli5
from eli5.lime import TextExplainer

te = TextExplainer(random_state=42)
te.fit(doc, pipe.predict_proba)
te.show_prediction(target_names=twenty_train.target_names)

/Users/mansur/Work/Projects/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [53]:
pipe.predict_proba([data])

array([[0.04625836, 0.39720962, 0.32938087, 0.22715115]])

In [89]:
#toxic_predictor = lambda doc: np.array(list(model.predict(doc).values())).T/np.array(list(model.predict(doc).values())).sum()

def toxic_predicton(doc):
    pred = np.array(list(model.predict(doc).values())).T
    out = pred/pred.sum()
    return out
    
toxic_predictor([data])


array([[3.91904309e-01, 6.74773280e-03, 2.65367997e-01, 3.64027057e-04,
        3.33725258e-01, 1.89067579e-03]])

In [90]:
toxic_predicton(data)

array([3.9190429e-01, 6.7477324e-03, 2.6536798e-01, 3.6402704e-04,
       3.3372524e-01, 1.8906757e-03], dtype=float32)

In [91]:
te = TextExplainer(random_state=42)

In [92]:
te.fit(data, toxic_predictor)

ValueError: probabilities do not sum to 1

In [13]:
import json
from detoxify import Detoxify
import datetime
import time

model = Detoxify('original', device='cpu')

with open("data/user_data.json", "r") as file_handler:    
    userdata = json.load(file_handler)

#preds = []
#for post in userdata['posts']:
#    result = model.predict(post['message'])
#    result = {str(key): value.round(3) for key, value in result.items()}
#    preds.append(result)

    

In [33]:
for i, post in enumerate(userdata['posts']):
    pred_post = model.predict(post['message'])
    pred_post = {str(key): value.round(3) for key, value in pred_post.items()}
    userdata['posts'][i]['pred'] = pred_post
    if post['has_comments']:
        for j, comment in enumerate(post['comments']):
            pred_comment = model.predict(comment['message'])
            pred_comment = {str(key): value.round(3) for key, value in pred_comment.items()}
            userdata['posts'][i]['comments'][j]['pred'] = pred_comment


In [58]:
#add toxicity prediction to userdata
for i, post in enumerate(userdata['posts']):
    pred_post = model.predict(post['message'])
    pred_post = {str(key): value.round(3) for key, value in pred_post.items()}
    #sort dictionary
    pred_post = dict(sorted(pred_post.items(), key=lambda item: item[1], reverse=True))
    userdata['posts'][i]['pred'] = pred_post
    
    if post['has_comments']:
        for j, comment in enumerate(post['comments']):
            pred_comment = model.predict(comment['message'])
            pred_comment = {str(key): value.round(3) for key, value in pred_comment.items()}
            userdata['posts'][i]['comments'][j]['pred'] = pred_comment
            userdata['posts'][i]['comments'][j]['id'] = str(i)+"_"+str(j)

In [12]:
pred_post = model.predict('this is toxic!')
pred_post = dict(sorted(pred_post.items(), key=lambda item: item[1], reverse=True))


for i, (key, value), in enumerate(pred_post.items()):
    print(i,key, value)
    
def other_prediction_button_position(pred_post, thres=0.1):
    out = np.argwhere(np.insert(np.diff((np.array(list(pred_post.values())) <=thres )+0), 0, 0)== 1).flatten()[0] + 1
    return out

(np.array(list(pred_post.values())) > 0.7).sum()

0 toxicity 0.16357347
1 obscene 0.0042379154
2 insult 0.0016475313
3 threat 0.00087232096
4 identity_hate 0.000684966
5 severe_toxicity 0.00033340332


0

In [74]:

flags = np.insert(0, np.diff(np.array(list(pred_post.values())) <=0.1 ), 0)
flags

/var/folders/d5/r2v0z0z17nnfzysx1xgwf9jc0000gn/T/ipykernel_20121/4205760616.py:1: FutureWarning: in the future insert will treat boolean arrays and array-likes as a boolean index instead of casting it to integer
  flags = np.insert(0, np.diff(np.array(list(pred_post.values())) <=0.1 ), 0)


array([0, 0, 0, 0, 0, 0])

In [36]:
test = {"id": uniq_id, 
        "message":b, 
        "user_fullname": a, 
        "avatar_img": 'https://i.pravatar.cc/100', 
        "timestamp": date_text, 
        "has_img": False,
        "post_img": '',
       "has_comments": False,
       "comments": []}
test

NameError: name 'uniq_id' is not defined

In [107]:
now = datetime.datetime.now()
date_text = now.strftime("%d %B %Y at %H:%M")

In [109]:
tstamp = datetime.datetime.timestamp(now)
print("timestamp: '"+str(tstamp)+ "',", "date_text: '"+now.strftime("%d %B %Y at %H:%M")+"'")
#datetime.datetime.fromtimestamp(tstamp)

timestamp: '1636936579.133466', date_text: '14 November 2021 at 19:36'


In [111]:
userdata

{'user_pagetitle': 'Social Media',
 'user_fullname': '24679 Student',
 'avatar_img': 'https://i.pravatar.cc/151',
 'stories': [{'id': 1,
   'message': 'story a',
   'bg_img': 'https://picsum.photos/600/301',
   'avatar_img': 'https://i.pravatar.cc/150'},
  {'id': 2,
   'message': 'story b',
   'bg_img': 'https://picsum.photos/601/302',
   'avatar_img': 'https://i.pravatar.cc/149'}],
 'posts': [{'id': 1,
   'user_fullname': 'A person',
   'avatar_img': 'https://i.pravatar.cc/152',
   'date_text': '14 November 2021 at 19:28',
   'timestamp': '1636936130.866307',
   'message': 'Im sick of all the politically correct rhetoric going out there. Stop spewing words like Mad Men, Orange is the New Black',
   'has_img': True,
   'post_img': 'https://picsum.photos/401/300',
   'has_comments': True,
   'comments': [{'id': 1,
     'user_fullname': 'A friend',
     'time_ago': '14h',
     'avatar_img': 'https://i.pravatar.cc/153',
     'message': 'This is a comment'},
    {'id': 2,
     'user_fullna

In [127]:
newlist = 
newlist

[{'id': 3,
  'user_fullname': 'A friend',
  'avatar_img': 'https://i.pravatar.cc/157',
  'timestamp': '1636936579.133466',
  'date_text': '14 November 2021 at 19:36',
  'message': 'Really hate the people around here!',
  'has_img': False,
  'post_img': '',
  'has_comments': False,
  'comments': []},
 {'id': 2,
  'user_fullname': 'Random person',
  'avatar_img': 'https://i.pravatar.cc/155',
  'timestamp': '1636936446.753207',
  'date_text': '14 November 2021 at 19:34',
  'message': 'You are not toxic!!',
  'has_img': False,
  'post_img': '',
  'has_comments': True,
  'comments': [{'id': 1,
    'user_fullname': 'A person',
    'time_ago': '14h',
    'avatar_img': 'https://i.pravatar.cc/156',
    'message': 'This is a comment'},
   {'id': 1,
    'user_fullname': 'A person #2',
    'time_ago': '14h',
    'avatar_img': 'https://i.pravatar.cc/161',
    'message': 'This is a comment'}]},
 {'id': 1,
  'user_fullname': 'A person',
  'avatar_img': 'https://i.pravatar.cc/152',
  'date_text': '14 

In [14]:
user_fullname = "A user"
avatar_img = "https://i.pravatar.cc/138"
post_id = "1"
message = "A new comment added"
now = datetime.datetime.now()
uniq_id = str(time.time()).replace('.','')

out = {"id": uniq_id, 
        "message":message, 
        "user_fullname": user_fullname, 
        "avatar_img": 'https://i.pravatar.cc/100', 
        "time_ago": "1 minute"}



In [15]:
out

{'id': '16381872099488618',
 'message': 'A new comment added',
 'user_fullname': 'A user',
 'avatar_img': 'https://i.pravatar.cc/100',
 'time_ago': '1 minute'}

In [16]:
with open("static/json/post_"+post_id+".json", "r") as file_handler:    
    userdata = json.load(file_handler)       
    userdata['comments'].append(out)



In [17]:
userdata

{'id': 1,
 'user_fullname': 'A person',
 'avatar_img': 'https://i.pravatar.cc/152',
 'date_text': '14 November 2021 at 19:28',
 'timestamp': '1636936130.866307',
 'message': 'Im sick of all the politically correct rhetoric going out there. Stop spewing words like Mad Men, Orange is the New Black',
 'has_img': True,
 'post_img': 'https://picsum.photos/401/300',
 'has_comments': True,
 'comments': [{'id': 1,
   'user_fullname': 'A friend',
   'time_ago': '14h',
   'avatar_img': 'https://i.pravatar.cc/153',
   'message': 'This is a comment'},
  {'id': 2,
   'user_fullname': 'Another friend',
   'time_ago': '12h',
   'avatar_img': 'https://i.pravatar.cc/148',
   'message': 'You are sick, man!'},
  {'id': '16381872099488618',
   'message': 'A new comment added',
   'user_fullname': 'A user',
   'avatar_img': 'https://i.pravatar.cc/100',
   'time_ago': '1 minute'}]}

In [18]:
with open("static/json/post_"+post_id+".json", "w") as file_handler:    
    json.dump(userdata, file_handler)

  

In [19]:
with open("static/json/post_"+post_id+".json", "r") as file_handler:    
    userdata = json.load(file_handler)       
userdata    



{'id': 1,
 'user_fullname': 'A person',
 'avatar_img': 'https://i.pravatar.cc/152',
 'date_text': '14 November 2021 at 19:28',
 'timestamp': '1636936130.866307',
 'message': 'Im sick of all the politically correct rhetoric going out there. Stop spewing words like Mad Men, Orange is the New Black',
 'has_img': True,
 'post_img': 'https://picsum.photos/401/300',
 'has_comments': True,
 'comments': [{'id': 1,
   'user_fullname': 'A friend',
   'time_ago': '14h',
   'avatar_img': 'https://i.pravatar.cc/153',
   'message': 'This is a comment'},
  {'id': 2,
   'user_fullname': 'Another friend',
   'time_ago': '12h',
   'avatar_img': 'https://i.pravatar.cc/148',
   'message': 'You are sick, man!'},
  {'id': '16381872099488618',
   'message': 'A new comment added',
   'user_fullname': 'A user',
   'avatar_img': 'https://i.pravatar.cc/100',
   'time_ago': '1 minute'}]}